# Fine-tune Llama 2 in Google Colab
> 🗣️ Large Language Model Course

❤️ Created by [@maximelabonne](https://twitter.com/maximelabonne), based on Younes Belkada's [GitHub Gist](https://gist.github.com/younesbelkada/9f7f75c94bdc1981c8ca5cc937d4a4da). Special thanks to Tolga HOŞGÖR for his solution to empty the VRAM.

This notebook runs on a T4 GPU. (Last update: 24 Aug 2023)


In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 bs4 evaluate rouge_score

In [2]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

from bs4 import BeautifulSoup

In [3]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [4]:
# The model that you want to train from the Hugging Face hub
model_name = "meta-llama/Llama-2-7b-chat-hf"

# The instruction dataset to use
dataset_name = "HuggingFaceH4/stack-exchange-preferences"

# Fine-tuned model name
new_model = "llama-2-7b-miniguanaco"

# Ed model name
ed_model = "llama-2-7b-miniguanaco-ed"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 8

# Alpha parameter for LoRA scaling
lora_alpha = 8

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of fine-tuning examples - StackOverflow
num_examples_sf = 100

# Number of fine-tuning examples - Ed
num_examples_ed = 100

# Number of training epochs
num_train_epochs = 5

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-5

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1 #num_train_epochs * num_examples_sf

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [5]:
new_model_name = f"{model_name.split('/')[-1]}_sf{num_examples_sf}_ed{num_examples_ed}_LoRAr{lora_r}_lr{learning_rate}"
new_model_name

'Llama-2-7b-chat-hf_sf100_ed100_LoRAr8_lr2e-05'

In [6]:
from huggingface_hub import login

login()

## train model and look at loss curves

In [7]:
# Preprocess Data
def get_first_ans(example):
    ans_soup = BeautifulSoup(example['answers'][0]['text'], 'html.parser')
    ans_text = ans_soup.get_text()
    q_soup = BeautifulSoup(example['question'], 'html.parser')
    q_text = q_soup.get_text()
    example['answers'] = ans_text
    example['question'] = q_text
    qa_text = f"<s>[INST] {q_text} [/INST] {ans_text}</s>"
    # example['QA'] = qa_text
    return {'QA': qa_text}

from functools import partial
from datasets import Dataset

def iterable_ds_to_ds(iterable_ds):
    def gen_from_iterable_dataset(iterable_ds):
        yield from iterable_ds
    return Dataset.from_generator(partial(gen_from_iterable_dataset, iterable_ds), features=iterable_ds.features)

In [ ]:
# Load dataset (you can process it here)
dataset = load_dataset(dataset_name, streaming=True, split='train')
dataset = dataset.map(
    get_first_ans,
    remove_columns=['qid', 'question', 'answers', 'date', 'metadata'])
dataset = iterable_ds_to_ds(dataset.take(100))

# Load Ed dataset
# ed_dataset = load_dataset("csv", data_files="EECS101DatasetSheet_EdStemOnly.csv", split='train')
ed_dataset = load_dataset('gperona/EdDataset', split='train')
ed_dataset = ed_dataset.train_test_split(test_size=0.1, seed=182)

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="QA",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

"""ed_training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)"""

# Set supervised fine-tuning parameters
'''ed_trainer = SFTTrainer(
    model=model,
    train_dataset=ed_dataset,
    peft_config=peft_config,
    dataset_text_field="QA",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)'''

# Train model
trainer.train()

# ed_trainer.train()


# Save trained model
trainer.model.save_pretrained(new_model_name)

Resolving data files:   0%|          | 0/758 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

In [16]:
%load_ext tensorboard
%tensorboard --logdir results/runs

'<s>[INST] I have been wanting to learn about 3D printing a long time so I really want this site to succeed but I have no previous experience with the subject. \nI was wondering how can I help the site at this early stage. I thought about asking about how to get started with 3D printing but SE explicitly discourages "easy" questions in the private beta.\nWhat can newbies like me do for the site at this stage besides voting questions and answers?\n [/INST] I would suggest doing a bit of basic research on 3D printing (including reading questions and answers).  From these you will learn more about it and hopefull you will have new questions about 3D printing that can be asked.  \nIf you are looking at getting a 3D printer, you could ask about different features listed and why they make prints better.\n</s>'

# Storing Model !!!! DONT RUN CELLS IN THIS SECTION (PRESS THE THING THAT CLOSES IT)
-Models
  - r8_lr
  - r8_lr
  - r8_lr
  - r8_lr


In [ ]:
! git clone https://huggingface.co/gperona/ed_data_model

In [ ]:
# storing model
model = base_model
from transformers import T5ForConditionalGeneration

model = T5ForConditionalGeneration.from_pretrained("t5-small")
# model.save_pretrained("/Users/davebaba/Desktop/" + "model_r" + str(lora_r) + "_lr" + str(learning_rate), from_pt=True)
model.save_pretrained("ed_data_model/" + "model_r" + str(lora_r) + "_lr" + str(learning_rate), from_pt=True)

# enter name of model here
# ! transformers-cli add-new-model gperona/ed_data_model

In [ ]:
%cd ed_data_model/

In [ ]:
!git config --global user.name "gperona"

In [ ]:
! git add *
! git commit -m "add model /model_r8_lr0.0002/"
! git push

In [ ]:
! cd ed_data_model
! git add *
! git commit -m "add model /model_r8_lr0.0002/"
! git push

In [ ]:
! git add /Users/davebaba/Desktop/model_r8_lr0.0002/
! git commit -m "add model /model_r8_lr0.0002/"
! git push

# Evaluate Stuff, including ROUGE

In [22]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "Easy EECS upper-div I am taking a pretty heavy courseload next semester with research and a job but I also nee dto take an EECS upper div class. Does anybody know a relatively lower courseload upper div EECS class?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


In [23]:
import evaluate
rouge = evaluate.load('rouge')

In [24]:
predictions = ["hello there", "my name kenobi", "1"]
references = ["hello there", "general kenobi", "1"]
results = rouge.compute(predictions=predictions, references=references)
print(results)

In [25]:
test_answers = []
for i in range(len(ed_dataset["test"]["Q"])):
    print(i)
    prompt = ed_dataset["test"]["Q"][i]
    pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=500)
    result = pipe(f"<s>[INST] {prompt} [/INST]")
    pred = [result['generated_text'].split("[/INST]")[1]]
    label = [ed_dataset["test"]["A"][i]]
    rouges = rouge.compute(predictions=pred, references=label)
    test_answers.append((prompt, result[0]['generated_text'], rouges['rouge1'], rouges['rouge2'], rouges['rougeL'], rouges['rougeLsum']))

TypeError: ignored

In [ ]:
result

In [ ]:
import csv
with open(f'ed_dataset_results_model_r{lora_r}_lr{learning_rate:.1e}.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    for row in test_answers:
        writer.writerow(row)


In [ ]:
# Empty VRAM
del model
del pipe
del trainer
import gc
gc.collect()
gc.collect()

In [ ]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
del base_model

In [ ]:
# Set up Ed data trainer
ed_dataset = load_dataset("csv", data_files="eecs_101_dataset.csv", split='train')

ed_trainer = SFTTrainer(
    model=model,
    train_dataset=ed_dataset,
    peft_config=peft_config,
    dataset_text_field="QA",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
ed_trainer.train()

# Save trained model
ed_trainer.model.save_pretrained(ed_model)

In [ ]:
# Empty VRAM
del model
del pipe
del ed_trainer
import gc
gc.collect()
gc.collect()

In [ ]:
# Reload model in FP16 and merge it with LoRA weights
new_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
ft_model = PeftModel.from_pretrained(new_model, ed_model)
ft_model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
prompt = dataset["train"]["Q"][0]
pipe = pipeline(task="text-generation", model=ed_model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

In [ ]:
!huggingface-cli login

model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)

## store results into a csv
as tag::question::response::rouge \
titled ed_dataset_results_r\<lora_r\>_lr\<learning_rate\>